In [209]:
import pandas as pd
import os
from datetime import datetime, timedelta
import shutil
import time
import pyperclip
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import warnings


import re

from datetime import datetime
from dateutil.relativedelta import relativedelta

# 기본 정리

## 함수

In [210]:
def calculate_difference_and_weekday(current_year, current_month, current_day, input_date):
    current_date = datetime(current_year, current_month, current_day)

    # 8자리 숫자를 연, 월, 일로 분리
    target_year = input_date // 10000
    target_month = (input_date % 10000) // 100
    target_day = input_date % 100
    months_difference= current_month - target_month
    target_date = datetime(target_year, target_month, target_day)

    # 현재 날짜와 입력된 날짜 간의 차이 계산
    delta = relativedelta(current_date, target_date)
    deltaday = delta.days +1

    return months_difference, deltaday, target_day


## Rawdata 생성할 날짜

In [211]:
# 입력 날짜 (8자리 숫자)
input_date = 20240202

# 현재 날짜 가져오기
current_date = datetime.now()

# 어제 날짜 계산
yesterday_date = current_date - timedelta(days=1)

# 어제 날짜의 년, 월, 일 추출
current_year = yesterday_date.year
current_month = yesterday_date.month
current_day = yesterday_date.day

months_difference, deltaday, target_day = calculate_difference_and_weekday(current_year, current_month, current_day, input_date)


print(f"네이버 기본 세팅에서 {input_date}은 {months_difference}달 전입니다. 입력날짜는 {target_day}입니다.")



네이버 기본 세팅에서 20240202은 0달 전입니다. 입력날짜는 2입니다.


## 디렉토리 정리

In [212]:
# 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()

# downloads 상대경로를 절대경로로 변환
relative_path_downloads = r'..\\..\\..\\Downloads'
download_path = os.path.join(current_directory, relative_path_downloads)

# "preprocessed" 상대경로를 절대경로로 변환
preprocessed_path = os.path.join(download_path, 'preprocessed')

# Check if the "preprocessed" folder exists
if os.path.exists(preprocessed_path):
    print(f"The 'preprocessed' folder already exists at: {preprocessed_path}")
else:
    # "preprocessed" 폴더 생성
    os.makedirs(preprocessed_path)


# ContentsList 상대경로를 절대경로로 변환
relative_path_contentslist = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\ContentsList.xlsx'
contentslist_path = os.path.join(current_directory, relative_path_contentslist)

# DailyRawdata
path_dailyrawdata = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\DailyRawdata'
file_name = f'blog_{input_date}.xlsx'
rawdata_file_name = os.path.abspath(os.path.join(current_directory, path_dailyrawdata, file_name))



The 'preprocessed' folder already exists at: c:\Users\ch.kang\Documents\GitHub\funnel_test\..\\..\\..\\Downloads\preprocessed


## ContentsList -> article_id -> bm, bh 구분

In [213]:
# 'ContentsList.xlsx' 파일 불러오기
df = pd.read_excel(contentslist_path)

# Filter the DataFrame to select rows where the label column starts with 'BM'
selected_rows_bm = df.loc[df['label'].str.startswith('BM')]

# Extract the article_ids from the 'node' column
extracted_values_bm = selected_rows_bm['node'].str[-12:].tolist()
article_ids_bm = [value for value in extracted_values_bm if value.isdigit()]

# Remove duplicates and sort the list
article_ids_bm = list(set(article_ids_bm))
article_ids_bm.sort()

# Filter the DataFrame to select rows where the first column starts with 'BH'
selected_rows_bh =df.loc[df['label'].str.startswith('BH')]

# Extract the article_ids from the 'node' column
extracted_values_bh = selected_rows_bh['node'].str[-12:].tolist()
article_ids_bh = [value for value in extracted_values_bh if value.isdigit()]

# Remove duplicates and sort the list
article_ids_bh = list(set(article_ids_bh))
values_to_remove = ['223203815685', '223253748515', '223267541556']
article_ids_bh = [id_bh for id_bh in article_ids_bh if id_bh not in values_to_remove]
article_ids_bm


['223203815685',
 '223208249149',
 '223209462852',
 '223212353603',
 '223215558503',
 '223229049237',
 '223240007276',
 '223241084754',
 '223247154141',
 '223250496328',
 '223253813865',
 '223258887581',
 '223261110315',
 '223264200872',
 '223267459993',
 '223267541556',
 '223270759875',
 '223273728446',
 '223278027001',
 '223282251484',
 '223283634476',
 '223286851479',
 '223289322164',
 '223291508141',
 '223293600922',
 '223297512585',
 '223298427705',
 '223299361549',
 '223300662460',
 '223304712190',
 '223306604658',
 '223320523083']

# MSP BLOG

In [214]:
## open Chrome -> naver -> login
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'qli_2018'  # Your ID
    npw = '12341234!@'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://admin.blog.naver.com/mysecondplace/stat/today')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

In [215]:
months_difference

0

In [216]:
# 빈 리스트를 생성하여 각각의 DataFrame을 저장할 준비
dfs = []
article_data_msp = []

# 게시글 정보 수집
for article_id in article_ids_bm:
    # Function to get total views, likes, and comments using Selenium
    url = f"https://blog.stat.naver.com/blog/article/{article_id}/cv"
    browser.get(url)

    # Click necessary buttons
    # target 날짜로 가서 해당 날짜 조회수 정보 얻기.

    ## target 날짜만큼 뒤로가기 버튼 클릭
    for _ in range(int(deltaday)):
        time.sleep(0.5)
        browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_info_section__1U7kD > div > div:nth-child(1) > div > a").click()
        time.sleep(0.5)
    time.sleep(2)
    # Wait until the view counts, like counts, and comment counts are loaded
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong"))
    )

    # Extract total views, total likes, and total comments using browser.find_element
    total_views_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(1) > strong")
    total_likes_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(1) > strong")
    total_comments_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(2) > strong")
    view_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong")
#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong

    total_views_text =  total_views_element.text.strip()
    total_likes_text = total_likes_element.text.strip()
    total_comments_text = total_comments_element.text.strip()
    view_text = view_element.text.strip()
    # Get the text values from the elements
    if view_text:
        view_numeric = int(view_text.replace(',', ''))  # Remove commas and convert to int
    else:
        view_numeric = 0  # Set a default value or handle it based on your requirements
    article_data_msp.append({
        'article_id': article_id,
        'node': f'https://blog.naver.com/mysecondplace/{article_id}/cv', 
        'total_views': total_views_text,
        'total_likes': total_likes_text,
        'total_comments': total_comments_text,
        'view': view_numeric
    })

    time.sleep(2)

    # 기타 작업 및 다운로드 프로세스(다운로드 클릭(페이지 이동))
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)"))).click()
    # 기타 작업 및 다운로드 프로세스(유입분석 클릭)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label"))).click()
    #datepicker 선택
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > button"))).click()
    # months_difference 만큼 이전달 버튼 클릭 반복
    for _ in range(int(months_difference)):
        WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > div:nth-child(4) > div > div > div.react-datepicker__month-container > div > a.react-datepicker__navigation.react-datepicker__navigation--previous.ui-datepicker-prev > span"))).click()

    # XPath를 사용하여 타겟 날짜 요소 고르기
    xpath_selector = f'//a[@class="react-datepicker__day--text ui-state-default" and text()="{target_day}"]'

    # 해당 XPath를 사용하여 날짜를 클릭
    day_element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_selector)))
    day_element.click()

    # 파일 다운로드 버튼 클릭
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green"))).click()

    # 파일 다운로드 완료를 기다림
    time.sleep(2.5)  # 실제로는 다운로드 완료를 확인하는 더 견고한 방법을 사용하세요.
    files_in_directory = [f for f in os.listdir(download_path) if f.endswith('.xlsx')]
    
    if files_in_directory:
        latest_file = max(files_in_directory, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
        new_file_name = f'{article_id}_{input_date}.xlsx'
        os.rename(os.path.join(download_path, latest_file), os.path.join(download_path, new_file_name))
        shutil.move(os.path.join(download_path, new_file_name), os.path.join(preprocessed_path, new_file_name))


        # Construct the path to the Excel file for the current article_id
        excel_file_path = os.path.join(preprocessed_path, new_file_name)

        try:
            # Read the Excel file into a DataFrame
            df1 = pd.read_excel(excel_file_path, header=8)
            # Extract date_info from the file name and store it
            df1['date_info'] = input_date
            df1['node'] = f'https://blog.naver.com/mysecondplace/{article_id}'
            df1['view'] = view_numeric
            # 현재의 DataFrame을 리스트에 추가
            dfs.append(df1)
        
        except Exception as e:
            print(f"Error processing file: {excel_file_path}")
            print(e)
browser.close()


    
# 모든 DataFrame을 결합하여 하나의 DataFrame으로 만듦
combined_df_msp = pd.concat(dfs, ignore_index=True)

# Rename columns
combined_df_msp.rename(columns={
    '유입경로': 'before_medium',
    '비율': 'before_medium_per',
    '상세유입경로': 'before',
    '비율.1': 'before_per'
}, inplace=True)

# Convert 'before_per' and 'before_medium_per' columns to numeric, coercing non-numeric values to NaN
combined_df_msp['before_per'] = pd.to_numeric(combined_df_msp['before_per'], errors='coerce')
combined_df_msp['before_medium_per'] = pd.to_numeric(combined_df_msp['before_medium_per'], errors='coerce')

# Round 'before_per' and 'before_medium_per' columns to 2 decimal places
combined_df_msp['before_per'] = combined_df_msp['before_per'].round(2)
combined_df_msp['before_medium_per'] = combined_df_msp['before_medium_per'].round(2)

# Add 'before_count' column
combined_df_msp['before_count'] = (combined_df_msp['view'] * (combined_df_msp['before_per'] / 100)).round(1)

# Add 'before_medium_count' column
combined_df_msp['before_medium_count'] = (combined_df_msp['view'] * (combined_df_msp['before_medium_per'] / 100)).round(1)

# Reorder columns
df_msp = combined_df_msp[['node', 'date_info', 'view', 'before', 'before_per', 'before_count', 'before_medium', 'before_medium_per', 'before_medium_count']]



c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, a

# HANPRO BLOG

In [217]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'hanpro911'  # Your ID
    npw = 'Zmfflwntlr1!'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://admin.blog.naver.com/hanpro911/stat/today')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

In [218]:
# 빈 리스트를 생성하여 각각의 DataFrame을 저장할 준비
dfs = []
article_data_han = []

# 게시글 정보 수집
for article_id in article_ids_bh:
    # Function to get total views, likes, and comments using Selenium
    url = f"https://blog.stat.naver.com/blog/article/{article_id}/cv"
    browser.get(url)

    # Click necessary buttons
    # target 날짜로 가서 해당 날짜 조회수 정보 얻기.

    ## target 날짜만큼 뒤로가기 버튼 클릭
    for _ in range(int(deltaday)):
        time.sleep(0.5)
        browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_info_section__1U7kD > div > div:nth-child(1) > div > a").click()
        time.sleep(0.5)
    time.sleep(2)
    # Wait until the view counts, like counts, and comment counts are loaded
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong"))
    )

    # Extract total views, total likes, and total comments using browser.find_element
    total_views_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(1) > strong")
    total_likes_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(1) > strong")
    total_comments_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(2) > strong")
    view_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong")
#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong

    total_views_text =  total_views_element.text.strip()
    total_likes_text = total_likes_element.text.strip()
    total_comments_text = total_comments_element.text.strip()
    view_text = view_element.text.strip()
    # Get the text values from the elements
    if view_text:
        view_numeric = int(view_text.replace(',', ''))  # Remove commas and convert to int
    else:
        view_numeric = 0  # Set a default value or handle it based on your requirements
    article_data_han.append({
        'article_id': article_id,
        'node': f'https://blog.naver.com/hanpro911/{article_id}/cv', 
        'total_views': total_views_text,
        'total_likes': total_likes_text,
        'total_comments': total_comments_text,
        'view': view_numeric
    })

    time.sleep(2)

    # 기타 작업 및 다운로드 프로세스(다운로드 클릭(페이지 이동))
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)"))).click()
    # 기타 작업 및 다운로드 프로세스(유입분석 클릭)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label"))).click()
    #datepicker 선택
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > button"))).click()
    # months_difference 만큼 이전달 버튼 클릭 반복
    for _ in range(int(months_difference)):
        WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > div:nth-child(4) > div > div > div.react-datepicker__month-container > div > a.react-datepicker__navigation.react-datepicker__navigation--previous.ui-datepicker-prev > span"))).click()

    # XPath를 사용하여 타겟 날짜 요소 고르기
    xpath_selector = f'//a[@class="react-datepicker__day--text ui-state-default" and text()="{target_day}"]'

    # 해당 XPath를 사용하여 날짜를 클릭
    day_element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_selector)))
    day_element.click()

    # 파일 다운로드 버튼 클릭
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green"))).click()

    # 파일 다운로드 완료를 기다림
    time.sleep(2.5)  # 실제로는 다운로드 완료를 확인하는 더 견고한 방법을 사용하세요.
    files_in_directory = [f for f in os.listdir(download_path) if f.endswith('.xlsx')]
    
    if files_in_directory:
        latest_file = max(files_in_directory, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
        new_file_name = f'{article_id}_{input_date}.xlsx'
        os.rename(os.path.join(download_path, latest_file), os.path.join(download_path, new_file_name))
        shutil.move(os.path.join(download_path, new_file_name), os.path.join(preprocessed_path, new_file_name))


        # Construct the path to the Excel file for the current article_id
        excel_file_path = os.path.join(preprocessed_path, new_file_name)

        try:
            # Read the Excel file into a DataFrame
            df1 = pd.read_excel(excel_file_path, header=8)
            # Extract date_info from the file name and store it
            df1['date_info'] = input_date
            df1['node'] = f'https://blog.naver.com/hanpro911/{article_id}'
            df1['view'] = view_numeric
            # 현재의 DataFrame을 리스트에 추가
            dfs.append(df1)
        
        except Exception as e:
            print(f"Error processing file: {excel_file_path}")
            print(e)


browser.close()
    
# 모든 DataFrame을 결합하여 하나의 DataFrame으로 만듦
combined_df_han = pd.concat(dfs, ignore_index=True)

# browser.close()

# Rename columns
combined_df_han.rename(columns={
    '유입경로': 'before_medium',
    '비율': 'before_medium_per',
    '상세유입경로': 'before',
    '비율.1': 'before_per'
}, inplace=True)

# Convert 'before_per' and 'before_medium_per' columns to numeric, coercing non-numeric values to NaN
combined_df_han['before_per'] = pd.to_numeric(combined_df_han['before_per'], errors='coerce')
combined_df_han['before_medium_per'] = pd.to_numeric(combined_df_han['before_medium_per'], errors='coerce')

# Round 'before_per' and 'before_medium_per' columns to 2 decimal places
combined_df_han['before_per'] = combined_df_han['before_per'].round(2)
combined_df_han['before_medium_per'] = combined_df_han['before_medium_per'].round(2)

# Add 'before_count' column
combined_df_han['before_count'] = (combined_df_han['view'] * (combined_df_han['before_per'] / 100)).round(1)

# Add 'before_medium_count' column
combined_df_han['before_medium_count'] = (combined_df_han['view'] * (combined_df_han['before_medium_per'] / 100)).round(1)

# Reorder columns
df_han = combined_df_han[['node', 'date_info', 'view', 'before', 'before_per', 'before_count', 'before_medium', 'before_medium_per', 'before_medium_count']]

c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ch.kang\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, a

## Output Preprocessed Data

In [219]:
df_result = pd.concat([df_msp, df_han])

# Save the DataFrame to the Excel file with the constructed file name
df_result.to_excel(rawdata_file_name, index=False)

# update total in statisics

In [220]:
# 기존 statistics 파일 불러오기
contentslist_path = os.path.join(current_directory, relative_path_contentslist)
df_stat = pd.read_excel(contentslist_path)

In [221]:
#article_data 모아오기
state_msp=pd.DataFrame(article_data_msp)
state_han=pd.DataFrame(article_data_han)
df_state= pd.concat([state_msp, state_han])


In [222]:
df_state['node'] = df_state['node'].str.replace('/cv', '')
# df_stat과 df_state를 contents_url을 기준으로 left join 수행
df_stat_updated = pd.merge(df_stat, df_state, on='node', how='left')

In [223]:
# 선택할 열 목록
selected_columns = ['label', 'node', 'category', 'board1', 'board2', 'title', 'post_start', 'post_end', 'total_views_y', 'total_likes_y', 'total_comments_y', 'view_y']

# 열 선택
df_stat_updated_selected = df_stat_updated[selected_columns]

# 선택한 열 중 '_y'를 제거하고 컬럼명을 변경
df_stat_updated_selected = df_stat_updated_selected.rename(columns=lambda x: x.replace('_y', ''))

# 결과 확인
df_stat_updated_selected



,label,node,category,board1,board2,title,post_start,post_end,total_views,total_likes,total_comments,view
0,BH-0001,https://blog.naver.com/hanpro911,blog_hanpro911,main,main,"구매하기, 마이세컨플레이스 (feat. 한프로) : 네이버 블로그",20220103,NaN,NaN,NaN,NaN,NaN
1,BH-0002,https://blog.naver.com/hanpro911/222611549543,blog_hanpro911,야매 DIY,야매 IOT,전원주택 방범 CCTV 설치하기 #1 (1/2.. : 네이버블로그,20220103,NaN,528,3,1,0.0
2,BH-0003,https://blog.naver.com/hanpro911/222611724607,blog_hanpro911,세컨하우스 개발,구매하기,세컨하우스용 시골집 매매하기 #1 (1/3) .. : 네이버블로그,20220103,NaN,"4,874",11,9,6.0
3,BH-0004,https://blog.naver.com/hanpro911/222612556607,blog_hanpro911,야매 DIY,노천탕 만들기,최신 로우테크 화목 노천탕 만들기 #1 (1/.. : 네이버블로그,20220104,NaN,"1,721",5,0,2.0
4,BH-0005,https://blog.naver.com/hanpro911/222612760199,blog_hanpro911,야매 DIY,노천탕 만들기,최신 로우테크 화목 노천탕 만들기 #2 (2/.. : 네이버블로그,20220104,NaN,"1,936",5,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
654,SK-0420,웨버 바베큐그릴 47,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN
655,SK-0421,유가읍세컨하우스,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN
656,SK-0422,인천지역 세컨하우스 추천,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN
657,SK-0423,충청도 서까래 시골집 매매,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN


In [224]:
df_stat_updated_selected.to_excel(contentslist_path)